In [199]:
''' Jupyter Notebook is an incredibly powerful tool for interactively developing and presenting data science projects.  '''
''' It allows you to run code, display the results, and add explanations, formulas, and charts all in one place.  '''

''' Script that can check for duplicate entries for TASK documents, it would be great if we could do this for all WMx/OMx ES indices.'''
''' Modify your script to check for any process name by the below field(s) to see if any duplicate data exists. '''
print(f"Check the records if index has duplicate recores")

Check the records if index has duplicate recores


In [200]:
#!pip uninstall -y elasticsearch
#!pip install elasticsearch==7.13.0
''' To begin, you need to install pytest and ipytest, a tool designed to run pytest tests directly in Jupyter. Execute the following in a Jupyter cell '''
# !pip install pytest ipytest

In [201]:
# !pip freeze

In [202]:
!pip show elasticsearch

Name: elasticsearch
Version: 7.13.0
Summary: Python client for Elasticsearch
Home-page: https://github.com/elastic/elasticsearch-py
Author: Honza Král, Nick Lang
Author-email: honza.kral@gmail.com, nick@nicklang.com
License: Apache-2.0
Location: /home/biadmin/monitoring/jupyter_notebook/.venv/lib/python3.9/site-packages
Requires: certifi, urllib3
Required-by: 


In [203]:
from elasticsearch import Elasticsearch
import os
import json
import pandas as pd
# import pytest
import ipytest
from dotenv import load_dotenv

In [204]:
''' To begin, you need to install pytest and ipytest, a tool designed to run pytest tests directly in Jupyter. Execute the following in a Jupyter cell '''
ipytest.autoconfig()

In [205]:
''' pip install python-dotenv'''
load_dotenv() # will search for .env file in local folder and load variables 

True

In [206]:
def get_headers():
    ''' Elasticsearch Header '''
    return {
            'Content-type': 'application/json', 
            'Authorization' : '{}'.format(os.getenv('BASIC_AUTH')),
            'Connection': 'close'
    }

In [207]:
def get_es_instance(host):
    es_client = Elasticsearch(hosts="http://{}".format(host), headers=get_headers(), timeout=5,  verify_certs=False)
    return es_client

# es_obj_s_client = get_es_instance("localhost:9200")

In [208]:
# resp = es_obj_s_client.cluster.health()
# print(json.dumps(resp, indent=2))

In [209]:
es_host_duplicates = {}

In [210]:
def json_value_to_transform_trim(raw_json):
    ''' update value in the form of json format'''
    # print(f"raw_json : {raw_json}")
    def get_recursive_nested_all(d):
        # print(f"get_recursive_nested_all : {d}")
        if isinstance(d, list):
            for i in d:
                get_recursive_nested_all(i)
        elif isinstance(d, dict):
            for k, v in d.items():
                if not isinstance(v, (list, dict)):
                    # print("%%%%", k, v)
                    d[k] = v
                else:
                    # print("####", k, v)
                    get_recursive_nested_all(v)
        return d

    return get_recursive_nested_all(raw_json)

In [211]:
def query_handler(process, script_query):
    ''' check dsl for any duplicates '''
    '''https://stackoverflow.com/questions/53076349/script-writing-to-get-distinct-value-from-elasticsearch '''
    ''' https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-bucket-terms-aggregation.html#search-aggregations-bucket-terms-aggregation-script '''
    ''' So you can see in the results as how key are constructed (keys are unique). '''
    '''
    {
      "size": 0,
      "aggs": {
        "duplicates": {
          "terms": {
            "script": {
              #"source": "doc['TASKID'].value",
              "source": "doc['ORDERKEY.keyword'].value + params.param + doc['ORDERLINENO'].value + params.param + doc['SITEID.keyword'].value",
              "lang": "painless",
              "params": {
                "param": ","
              }
            },
            "min_doc_count": 2,
            "size": 10000
          }
        }
      }
    }
    '''
    query = {
          "size": 0,
          "aggs": {
            "duplicates": {
              "terms": {
                "script": {
                  # "source": "doc['TASKID'].value",
                  "source": script_query,
                  "lang": "painless",
                  "params": {
                    "param": ","
                  }
                },
                "min_doc_count": 2,
                "size" : 10000
              }
            }
          }
        }

    extract_ids_query = {
          "size": 0,
          "aggs": {
            "duplicates": {
              "terms": {
                "script": {
                  # "source": "doc['TASKID'].value",
                  "source": script_query,
                  "lang": "painless",
                  "params": {
                    "param": ","
                  }
                },
                "min_doc_count": 2,
                "size" : 10000
              },
              "aggs": {
                "first-report": {
                  "top_hits": {
                    "_source": ["_id"], 
                    "size": 10
                  }
                }
              }
            }
          }
        }
    # if process == 'sample':
        # query = {
        #   "size": 0,
        #   "aggs": {
        #     "duplicates": {
        #       "terms": {
        #             "field": "TASKID",
        #             "min_doc_count": 2,
        #             "size" : 10000  
        #         }
        #      }
        #   }
        # }

    # print(json.dumps(query, indent=2))
    # exit(1)
    
    return process, query

In [212]:
def check_duplicates_tasks(env, process_script_query_dict, process):
    ''' check duplicates'''
    
    # print('\n', env)
    # print(os.getenv("{}_ES_HOST".format(str(env).upper())))
    es_host = "{}_ES_HOST".format(str(env).upper())
    dataframe_column.append(es_host)
    dataframe_process.append(process)

    if 'wx' in process:
        dataframe_db.append('WMx DB')
    elif 'om' in process:
        dataframe_db.append('OMx DB')
    # print(es_host)
    
    ''' instance '''
    es_obj_s_client = get_es_instance(os.getenv(es_host))
    dataframe_es_client.append(es_obj_s_client)

    ''' return index_name and query '''
    index_name, query = query_handler(process, process_script_query_dict)
    
    response = es_obj_s_client.search(index=index_name, body=query)
    # print(json.dumps(response, indent=2))
    
    duplicates_list = response['aggregations']['duplicates']['buckets']
    
    # print(f"total duplicates : {json.dumps(len(duplicates_list), indent=2)}")
    # print(json.dumps(duplicates_list, indent=2))
    # lookup = json_value_to_transform_trim(response['aggregations'])
    # print(f"lookup - type(lookup) : {type(lookup)}, lookup : {lookup}")

    ''' total docs with len(buckets) + "sum_other_doc_count" '''
    '''
    if len(duplicates_list) > 0:
        AGGS_TOTAL_DOCS = len(duplicates_list) + int(response['aggregations']['duplicates']['sum_other_doc_count'])
    else:
        AGGS_TOTAL_DOCS = len(duplicates_list)
    '''
    AGGS_TOTAL_DOCS = len(duplicates_list)
    # print(env, process, len(duplicates_list), int(response['aggregations']['duplicates']['sum_other_doc_count']))
    # print(env, process, AGGS_TOTAL_DOCS, query, response)
    es_host_duplicates.update({es_host : AGGS_TOTAL_DOCS})
    dataframe_value.append(AGGS_TOTAL_DOCS)
    # dataframe_query_dsl.append(query)

    ''' save raw_data '''
    raw_env_dict[env].update({
        process : duplicates_list
    })

    # return dataframe_dict

In [213]:

from collections import defaultdict

# env_list = ['prod1','prod2','prod3','prod4','prod6','prod7','prod8','prod9','prod10','prod12','prod13','prod14','prod16','prod17','prod18','prod19','prod20']
# env_list = ['dev']
# env_list = ['prod1']
env_list = ['prod1','prod2']
dataframe_dict = {}
raw_env_dict = defaultdict(dict)
dataframe_column, dataframe_process, dataframe_es_client, dataframe_value, dataframe_db, dataframe_query_dsl = [], [], [], [], [], []

''' Script that can check for duplicate entries for TASK documents, it would be great if we could do this for all WMx/OMx ES indices.'''
''' Modify your script to check for any process name by the below field(s) to see if any duplicate data exists. '''

''' Create query dsl with unique fields '''
process_script_query_dict = {
    'wx_adjustment' : "doc['ADJUSTMENTKEY.keyword'].value + params.param + doc['SITEID.keyword'].value",
    'wx_asn' : "doc['ASNKEY.keyword'].value + params.param + doc['SITEID.keyword'].value",
    'wx_asndtl' : "doc['ASNKEY.keyword'].value + params.param + doc['ASNLINENO'].value + params.param + doc['SITEID.keyword'].value",
    'wx_task' : "doc['TASKID'].value",
    'wx_orderdtl' : "doc['ORDERKEY.keyword'].value + params.param + doc['ORDERLINENO'].value + params.param + doc['SITEID.keyword'].value",
    'wx_inv_hold' : "doc['INVHOLDID'].value",
    'wx_inv_holdtrans' : "doc['INVHOLDTRANSID'].value",
    'om_appointment' : "doc['CRN'].value"
}

for env in env_list:
    print(f"Progressing for {env} ..")
    for each_index in process_script_query_dict.keys():
        check_duplicates_tasks(env, process_script_query_dict.get(each_index), each_index)
# print('\n Duplicate records')
# print(json.dumps(dataframe_dict, indent=2))

Progressing for prod1 ..
Progressing for prod2 ..


In [214]:
''' update dict for dataframe '''
dataframe_dict.update({'ES_Cluster' : dataframe_column})
dataframe_dict.update({'ES URL' : dataframe_es_client})
dataframe_dict.update({'DB' : dataframe_db})
dataframe_dict.update({'Process_Name' : dataframe_process})
dataframe_dict.update({'Duplicates_Count' : dataframe_value})
# dataframe_dict.update({'Query DSL' : dataframe_query_dsl})

In [215]:
df = pd.DataFrame(dataframe_dict)
# display(df)

In [216]:
''' print out raw data '''
# print(json.dumps(raw_env_dict, indent=2))

In [217]:
# writing to Excel
# df.to_csv("duplicate.csv")
# print("Export csv files successfully..")

In [218]:
''' filter if any duplicate data exist in df dataframe '''
# display(df.filter(items=['ES_Cluster', 'Process_Name'], axis=1))

In [219]:
''' check if any duplicate data exist in df dataframe '''
df = df[df['Duplicates_Count'] > 0]
display(df)

,ES_Cluster,ES URL,DB,Process_Name,Duplicates_Count


In [220]:
def test_pytest_func():
    assert 42 == 42

In [221]:
def test_query_validate():
    ''' Validate the query '''
    index_name, query = query_handler('test', "doc['TASKID'].value")
    # print(query)
    assert index_name == 'test'
    assert type(query) == dict
    assert query == {
      "size": 0,
      "aggs": {
        "duplicates": {
          "terms": {
            "script": {
              "source": "doc['TASKID'].value",
              # "source": "doc['ORDERKEY.keyword'].value + params.param + doc['ORDERLINENO'].value + params.param + doc['SITEID.keyword'].value",
              "lang": "painless",
              "params": {
                "param": ","
              }
            },
            "min_doc_count": 2,
            "size": 10000
          }        }
      }
    }

In [222]:
''' Execute the tests using ipytest.run(). You can pass command-line arguments to control test behavior: '''
''' The pytest framework makes it easy to write small, readable tests, and can scale to support complex functional testing for applications and libraries. '''
ipytest.run('-vv')

======================================= test session starts ========================================
platform linux -- Python 3.9.0, pytest-8.3.4, pluggy-1.5.0 -- /home/biadmin/monitoring/jupyter_notebook/.venv/bin/python3.9
cachedir: .pytest_cache
rootdir: /home/biadmin/monitoring/jupyter_notebook
plugins: anyio-4.5.0
collecting ... collected 2 items

t_4ad4cffb718845b3915bc958fa7a6001.py::test_pytest_func PASSED                               [ 50%]
t_4ad4cffb718845b3915bc958fa7a6001.py::test_query_validate PASSED                            [100%]

======================================== 2 passed in 0.02s =========================================


<ExitCode.OK: 0>